In [96]:
from google.cloud import bigquery
import urllib.request
import urllib.parse
import json
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

load_dotenv()

apikey = os.getenv('FMP_API_KEY')
project_id = os.getenv('GCP_PROJECT_ID')

In [58]:
client = bigquery.Client(project=project_id)
db = 'stock_data'
table = 'raw_stock_data'
table_id = f"{project_id}.{db}.{table}"
table_id

'stock-prices-442513.stock_data.raw_stock_data'

In [97]:
symbols = ['AAPL', 'TTD', 'GOOG']

query_string = f"""
    WITH symbol_list AS (
        SELECT symbol
        FROM UNNEST(@symbols) AS symbol
    )
    SELECT symbol_list.symbol, MAX(stock_data.`date`) AS `date`
    FROM symbol_list
    LEFT JOIN `{table_id}` AS stock_data
    ON symbol_list.symbol = stock_data.`symbol`
    GROUP BY symbol_list.symbol
"""

job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter('symbols', 'STRING', symbols)
        ]
    )
query_job = client.query(query_string, job_config=job_config)
results = query_job.result()

In [ ]:
api_lookup = []
default_date = '2024-11-19'

for row in results:
    if row['date']:
        date_plus_1 = row['date'] + timedelta(days=1)
        r = [
            row['symbol'],
            datetime.strftime(date_plus_1, format='%Y-%m-%d')
        ]
    else:
        r = [
            row['symbol'],
            default_date
        ]

    api_lookup.append(r)

api_lookup

[['AAPL', '2024-11-23'], ['TTD', '2024-11-23'], ['GOOG', '2024-11-19']]

In [ ]:
def historical_url(apikey, ticker, from_date=None):
    url = 'https://financialmodelingprep.com/api/v3/historical-price-full/'

    query_params = {
        'apikey': apikey
    }

    if from_date is not None:
        query_params['from'] = from_date

    encoded_params = urllib.parse.urlencode(query_params)

    full_url = f"{url}{ticker}?{encoded_params}"

    return full_url

response = urllib.request.urlopen(historical_url(apikey, 'TTD', '2024-11-20'))

data = json.loads(response.read())

In [ ]:
def retrieve_data(apikey, symbol, from_date):
    url = historical_url(apikey, symbol, from_date)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data

In [ ]:
def process_data(apikey, api_lookup):
    for item in api_lookup:
        symbol = item[0],
        from_date = item[1]
        data = retrieve_data(apikey, symbol, from_date)
        for stock_data in data['historical']:
            stock_data['symbol'] = data['symbol']
            stock_data['timestamp'] = datetime.now().isoformat()
        rows_to_insert = stock_data['historical']
        errors = client.insert_rows_json(table_id, rows_to_insert)
        if errors == []:
            print("Data inserted successfully.")
        else:
            print("Encountered errors while inserting rows:", errors)
